In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output

In [2]:
TIEMPO_DE_ESPERA = 5.5
start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=3200x10000")
# options.headless = True
browser = webdriver.Chrome(options = chrome_options)

In [35]:
browser.save_screenshot('asd.png')

True

In [197]:
browser.get('https://oldnavy.gap.com/browse/category.do?cid=1017042&mlink=5151,,flyout_men_ShopbyCategory_Socks_Underwear_&clink=15682852')

In [6]:
str("2019-10-30")

'2019-10-30'

In [3]:
url_base = "https://oldnavy.gap.com"
ipath = f'OLDNAVY_LINKS2019-11-19.xlsx'
dfLinks = pd.read_excel(ipath)

In [4]:
dfLinks = dfLinks[dfLinks['sexo']=='Men']

In [5]:
dfLinks['sexo'].unique()

array(['Men'], dtype=object)

In [6]:
#modal de inicio
try:
    browser.execute_script("document.getElementsByClassName('universal-modal__close-button')[0].click();")
except:
    pass

In [7]:
#promociones_click para ocultar
try:
    browser.execute_script("document.getElementsByClassName('promoDrawer__handlebar__icon')[0].click();")
except:
    pass

In [8]:
dfLinks.reset_index(drop=True,inplace=True)

In [12]:
print(browser.current_url)

https://oldnavy.gap.com/browse/category.do?cid=72091&mlink=5151,,flyout_women_ShopbyCategory_Tops&clink=15682852


In [211]:
browser.save_screenshot('asd.png')

In [9]:
browser.get(row['url'])

NameError: name 'row' is not defined

In [231]:
dfLinks = dfLinks.loc[14:].copy()

In [228]:
lista_productos[-1]

['product346944062',
 'Printed Socks 3-Pack for Men',
 'Men',
 'Socks & Underwear ',
 ['$11.99$9.97'],
 'https://www1.assets-gap.com/webcontent/0017/117/647/cn17117647.jpg',
 'https://oldnavy.gap.com/browse/product.do?pid=346944062&cid=1083572&pcid=1017042&vid=1&grid=pds_12_13_1',
 'Multi-Packs ',
 'multi']

In [10]:
lista_productos = []
excepts = []
for index,row in dfLinks.iterrows():
    browser.get(row['url'])
    
#    browser.implicitly_wait(30)
    #"cantidad categorias"
    if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):
        continue
    
    try:
        #le asigno una clase "FILTRO_CATEGORIA" para poder buscar los colores
        time.sleep(5)
        category = [i for i in browser.find_elements_by_class_name('tabs--object') if i.text.upper() == 'CATEGORY'][0]
        cantidad_categorias = [i.get_attribute('name') for i in category.find_element_by_class_name('accordion--content--inner').find_elements_by_tag_name('input')]
    except:
        cantidad_categorias = ['sin categoria']
        excepts.append([row['url'],
                       row['tipo']])
        continue

    #FOR DE CANTIDAD DE CATEGORIAS
    for index_categoria,nombre_categoria in enumerate(cantidad_categorias):
        
        
        #CLICK EN LA CATEGORIA DEL INDICE

        time.sleep(3)
        try:
            browser.execute_script(f'''
            document.getElementsByClassName('tabs--object tabs--header-object facet-style')[0].getElementsByTagName('input')[{index_categoria}].click();
            ''')
        except:
            break
        
       
        if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):
            
            continue

        
        #"cantidad de colores"
        #le asigno una clase "FILTRO_COLOR" para poder buscar los colores
        time.sleep(5)
        try:
            
            aux_col = [i for i in browser.find_elements_by_class_name('tabs--object') if i.text.upper() == 'COLOR'][0]            
            cantidad_colores = [i.get_attribute('name') for i in aux_col.find_element_by_class_name('accordion--content--inner').find_elements_by_tag_name('input')]
        except:
            excepts.append(browser.current_url)
        
        
        #LOOP CANTIDAD DE COLORES
        for index_color,nombre_color in enumerate(cantidad_colores):

            #CLICK EN EL COLOR DEL INDICE
#            browser.implicitly_wait(30)
            time.sleep(5)
            
            try:
                browser.execute_script(f'''
                document.getElementsByClassName('tabs--object tabs--header-object facet-color')[0].getElementsByTagName('input')[{index_color}].click();
                ''')
            except:
                pass
        
        
            if '0 Items in the product grid' == browser.execute_script('''var x = document.getElementsByClassName('category__item-count')[0].textContent;return x;'''):

                continue
        

            #NOMBRE COLOR
            #CANTIDAD DE PRODUCTOS DESPUÉS DE FILTRAR
#            browser.implicitly_wait(30)
            time.sleep(5)
            try:
                cant_productos = browser.execute_script('''
                var x = document.getElementsByClassName('product-card-grid__root grid ism-root')[0].getElementsByClassName('product-card-grid__item-1-2 product-card-grid__item-lg-1-3 product-card-grid__item-xl-1-3 product-card-grid__item-1280-1-4').length;
                return x;''')
            except:
                continue
            
            html_plano = browser.execute_script('''
            var x = document.getElementsByClassName('product-card-grid__root grid ism-root')[0].innerHTML;
            return x''')
            html_plano = BeautifulSoup(html_plano)
            for html in html_plano:
                lista_productos.append([html.a['id'],
                                        html.a.img['alt'],
                                        row['sexo'],
                                        row['tipo'],
                                        [i.text for i in html.find_all(class_='product-card-price')],
                                        html.a.img['src'],
                                        html.a['href'],
                                        nombre_categoria,
                                        nombre_color])

            #BORRAR FILTRO COLOR
            try:
                browser.execute_script(f'''
                document.getElementsByClassName('tabs--object tabs--header-object facet-color')[0].getElementsByTagName('input')[{index_color}].click();
                ''')
            except:
                browser.execute_script('''
                document.getElementsByClassName('tag--icon icon-tag-x')[1].click();
                ''')
        
        
        #borrar filtro categoria
        try:
            
            #WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.CLASS_NAME, "FILTRO_CATEGORIA"))).find_elements_by_tag_name('input')[index_categoria].click()
            browser.execute_script(f'''
            document.getElementsByClassName('tag--icon icon-tag-x')[0].click();
            ''')
        except:
            try:
                browser.execute_script(f'''
                    document.getElementsByClassName('tabs--object tabs--header-object facet-style')[0].getElementsByTagName('input')[{index_categoria}].click();
                    ''')
            except:
                print("no borro filtro categoria")

In [218]:
browser.save_screenshot('asd.png')

True

In [219]:
print(browser.current_url)

https://oldnavy.gap.com/browse/category.do?cid=5249&mlink=5151,,flyout_men_ShopbyCategory_Tees_&clink=15682852#color=1029


In [11]:
df = pd.DataFrame(lista_productos)

In [12]:
df.rename(columns={0:'sku',
                  1:'descripcion',
                  2:'categoria',
                  3:'tipo',
                  4:'prices',
                  5:'img',
                  6:'url',
                  7:'sub_categoria',
                  8:'color'},inplace=True)

In [13]:
df['prices'] = df['prices'].apply(lambda x:  x[0])

df['precio_dto'] = df['prices'].apply(lambda x : x.split('$')[2] if len(x.split('$')) == 3 else x.split('$')[1])

df['precio_original'] = df['prices'].apply(lambda x : x.split('$')[1] if len(x.split('$')) == 3 else x.split('$')[1])

In [14]:
browser.quit()

In [15]:
df.to_excel(f'./Salida/OLDNAVY_MEN{fecha}.xlsx')

In [240]:
fecha

datetime.date(2019, 11, 12)

In [168]:
%hist

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options(